# Alejandro Prompt:

You are a data analytic consultant and the product team needs to segment customers based on their purchasing behavior for a new feature rollout. The columns of the data are: 

Data Schema: user_activity ,  user_id , last_login_date , feature_usage_count  , account_type  
Transactions :  transaction_id , user_id , transaction_date , amount , platform
User_preferences :  user_id , communication_preference , interface_theme , notification_settings 

please, don't forget to generate the data at the beggining of the code

We need to create in SQL (Sqlite) Active users (logged in last 30 days),  
Filter by high-value customers (top 20% by spending) and User preference trends for the identified customers

# Generated Code:

In [3]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 1. Generate Data
np.random.seed(42)
user_ids = [f"user_{i}" for i in range(1, 101)]

user_activity = pd.DataFrame({
    'user_id': user_ids,
    'last_login_date': [(datetime.now() - timedelta(days=np.random.randint(0, 60))).strftime('%Y-%m-%d') for _ in user_ids],
    'feature_usage_count': np.random.poisson(5, 100),
    'account_type': np.random.choice(['free', 'premium', 'enterprise'], size=100, p=[0.5, 0.4, 0.1])
})

transactions = pd.DataFrame({
    'transaction_id': [f"txn_{i}" for i in range(1, 301)],
    'user_id': np.random.choice(user_ids, size=300),
    'transaction_date': [(datetime.now() - timedelta(days=np.random.randint(0, 90))).strftime('%Y-%m-%d') for _ in range(300)],
    'amount': np.round(np.random.exponential(scale=50, size=300), 2),
    'platform': np.random.choice(['web', 'mobile'], size=300)
})

user_preferences = pd.DataFrame({
    'user_id': user_ids,
    'communication_preference': np.random.choice(['email', 'sms', 'push'], size=100),
    'interface_theme': np.random.choice(['dark', 'light'], size=100),
    'notification_settings': np.random.choice(['all', 'important only', 'none'], size=100)
})

# 2. Load Data into SQLite
conn = sqlite3.connect(':memory:')
user_activity.to_sql('user_activity', conn, index=False, if_exists='replace')
transactions.to_sql('transactions', conn, index=False, if_exists='replace')
user_preferences.to_sql('user_preferences', conn, index=False, if_exists='replace')
conn.commit()

active_users_query = """
SELECT user_id
FROM user_activity
WHERE DATE(last_login_date) >= DATE('now', '-30 days')
"""
active_users = pd.read_sql_query(active_users_query, conn)

high_value_query = """
WITH total_spending AS (
    SELECT user_id, SUM(amount) AS total_spent
    FROM transactions
    GROUP BY user_id
),
percentile_cutoff AS (
    SELECT PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY total_spent) AS cutoff
    FROM total_spending
)
SELECT t.user_id, t.total_spent
FROM total_spending t, percentile_cutoff p
WHERE t.total_spent >= p.cutoff
"""
# Note: SQLite does not support PERCENTILE_CONT natively, so we simulate it below.
spending = pd.read_sql_query("SELECT user_id, SUM(amount) AS total_spent FROM transactions GROUP BY user_id", conn)
cutoff = spending['total_spent'].quantile(0.8)
high_value_users = spending[spending['total_spent'] >= cutoff]

# Join with active users
active_high_value = pd.merge(high_value_users, active_users, on='user_id', how='inner')

# Join with preferences
preference_trends = pd.merge(active_high_value, user_preferences, on='user_id', how='left')

# Trend Summary
trend_summary = preference_trends[['communication_preference', 'interface_theme', 'notification_settings']] \
    .value_counts().reset_index(name='count')


print("🎯 Active High-Value Users:", len(active_high_value))
print("\n📊 Preference Trends for These Users:")
print(trend_summary.head())


🎯 Active High-Value Users: 9

📊 Preference Trends for These Users:
  communication_preference interface_theme notification_settings  count
0                     push            dark        important only      2
1                     push            dark                  none      2
2                      sms           light        important only      2
3                    email            dark                   all      1
4                     push            dark                   all      1


# Follow-up Prompts:

How were percentile calculations handled?
I asked to calculate the percentile and generate a plot for the new version
spending['percentile'] = spending['total_spent'].rank(pct=True) * 100
I alos asked the top ten with the highest purchasing behavior, so the sales team can make the strategy


What approaches to date filtering were used?

Filter: Active Users (Logged in Last 30 Days)
WHERE DATE(last_login_date) >= DATE('now', '-30 days')

Filter: Top 20% by Spending (High-Value Customers)
cutoff = spending['total_spent'].quantile(0.8)
high_value_users = spending[spending['total_spent'] >= cutoff]

Filter: Users Who Are Both Active AND High-Value
active_high_value = pd.merge(high_value_users, active_users, on='user_id', how='inner')

Filter: NaNs Filled for Transaction-Free Users
df[['total_spend', 'num_transactions']] = df[['total_spend', 'num_transactions']].fillna(0)



How was the query optimized, CTE, subquery etc…?
the LLM used two CTE
CTE 1 (total_spending): Pre-aggregates user-level spend to avoid repeating the GROUP BY clause.
CTE 2 (percentile_cutoff): Intended to calculate the 80th percentile using window function.

Filtering after joins
active_high_value = pd.merge(high_value_users, active_users, on='user_id', how='inner')

# Final Solution:

In [5]:
query = """
WITH total_spending AS (
    SELECT 
        user_id, 
        SUM(amount) AS total_spent
    FROM transactions
    GROUP BY user_id
),
active_users AS (
    SELECT 
        user_id
    FROM user_activity
    WHERE DATE(last_login_date) >= DATE('now', '-30 days')
),
active_spenders AS (
    SELECT 
        ts.user_id,
        ts.total_spent
    FROM total_spending ts
    INNER JOIN active_users au ON ts.user_id = au.user_id
),
ranked_spenders AS (
    SELECT 
        *,
        RANK() OVER (ORDER BY total_spent DESC) AS rank
    FROM active_spenders
)
SELECT 
    rs.user_id,
    rs.total_spent,
    ua.last_login_date,
    ua.account_type,
    up.communication_preference,
    up.interface_theme,
    up.notification_settings
FROM ranked_spenders rs
JOIN user_activity ua ON rs.user_id = ua.user_id
JOIN user_preferences up ON rs.user_id = up.user_id
WHERE rs.rank <= 10
ORDER BY rs.total_spent DESC;
"""

top10_users = pd.read_sql_query(query, conn)
top10_users

,user_id,total_spent,last_login_date,account_type,communication_preference,interface_theme,notification_settings
0,user_12,544.23,2025-05-25,premium,sms,light,important only
1,user_23,495.22,2025-05-12,premium,email,dark,all
2,user_55,475.55,2025-05-27,free,push,dark,important only
3,user_96,474.36,2025-05-22,premium,push,dark,none
4,user_37,386.13,2025-05-09,free,sms,light,important only
5,user_59,339.71,2025-05-11,enterprise,push,dark,all
6,user_29,284.51,2025-05-15,free,push,dark,none
7,user_74,273.90,2025-05-28,enterprise,push,dark,important only
8,user_82,237.04,2025-06-01,premium,sms,dark,important only
9,user_57,235.51,2025-05-18,premium,email,dark,all
